https://www.kaggle.com/c/house-prices-advanced-regression-techniques

Competition Description:

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Practice Skills:

Creative feature engineering Advanced regression techniques like random forest and gradient boosting

download data sets from the Kaggle and import csv files

In [3]:
%fs ls dbfs:/FileStore/tables/iz4qo8a01498148128281/

In [4]:
%python

train = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('dbfs:/FileStore/tables/iz4qo8a01498148128281/train.csv')
test = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('dbfs:/FileStore/tables/iz4qo8a01498148128281/test.csv')

display(train)

In [5]:
train.registerTempTable("train")
test.registerTempTable("test")

In [6]:
%sql show tables

Checking the data formats for each columns

In [8]:
train.printSchema()

In [9]:
print(train.count())
print(test.count())

In [10]:
%python
list(train)

Data transformation, merging tables, and cleaning

In [12]:
%scala
val train = sqlContext.table("train")
val test = sqlContext.table("test")

In [13]:
%scala
sqlContext.cacheTable("train")
sqlContext.cacheTable("test")

Converting categorical variables from string to numeric forms.

In [15]:
%scala
import org.apache.spark.ml.feature.StringIndexer 

val val1 = new StringIndexer().setInputCol("MSZoning").setOutputCol("MSZoning1") 
val val2 = new StringIndexer().setInputCol("Neighborhood").setOutputCol("Neighborhood1") 
val val3 = new StringIndexer().setInputCol("LotShape").setOutputCol("LotShape1") 
val val4 = new StringIndexer().setInputCol("LandContour").setOutputCol("LandContour1")
val val5 = new StringIndexer().setInputCol("LandSlope").setOutputCol("LandSlope1") 
val val6 = new StringIndexer().setInputCol("PavedDrive").setOutputCol("PavedDrive1")
val val7 = new StringIndexer().setInputCol("Condition2").setOutputCol("Condition21") 
val val8 = new StringIndexer().setInputCol("ExterCond").setOutputCol("ExterCond1")
val val9 = new StringIndexer().setInputCol("BsmtQual").setOutputCol("BsmtQual1") 
val val10 = new StringIndexer().setInputCol("KitchenQual").setOutputCol("KitchenQual1")
val val11 = new StringIndexer().setInputCol("RoofMatl").setOutputCol("RoofMatl1") 
val val12 = new StringIndexer().setInputCol("CentralAir").setOutputCol("CentralAir1")
val val13 = new StringIndexer().setInputCol("Functional").setOutputCol("Functional1")
val val14 = new StringIndexer().setInputCol("FireplaceQu").setOutputCol("FireplaceQu1")
val val15 = new StringIndexer().setInputCol("GarageType").setOutputCol("GarageType1") 
val val16 = new StringIndexer().setInputCol("GarageFinish").setOutputCol("GarageFinish1")
val val17 = new StringIndexer().setInputCol("GarageQual").setOutputCol("GarageQual1") 
val val18 = new StringIndexer().setInputCol("GarageCond").setOutputCol("GarageCond1")
val val19 = new StringIndexer().setInputCol("SaleCondition").setOutputCol("SaleCondition1") 

val train1 = val1.fit(train).transform(train)
val train2 = val2.fit(train1).transform(train1)
val train3 = val3.fit(train2).transform(train2)
val train4 = val4.fit(train3).transform(train3)
val train5 = val5.fit(train4).transform(train4)
val train6 = val6.fit(train5).transform(train5)
val train7 = val7.fit(train6).transform(train6)
val train8 = val8.fit(train7).transform(train7)
val train9 = val9.fit(train8).transform(train8)
val train10 = val10.fit(train9).transform(train9)
val train11 = val11.fit(train10).transform(train10)
val train12 = val12.fit(train11).transform(train11)
val train13 = val13.fit(train12).transform(train12)
val train14 = val14.fit(train13).transform(train13)
val train15 = val15.fit(train14).transform(train14)
val train16 = val16.fit(train15).transform(train15)
val train17 = val17.fit(train16).transform(train16)
val train18 = val18.fit(train17).transform(train17)
val train19 = val19.fit(train18).transform(train18)




// indexed.show()
// indexed.printSchema()
display(train19)
train19.registerTempTable("trainCat")

// Qurious how the 'MSZoning' values below are not numerical but still showing the string values.

In [16]:
%sql show tables

In [17]:
%sql SELECT * FROM traincat limit 30


In [18]:
%sql
show columns from traincat

Creating a new table with variabels that are selected from a preliminary variable selection step.  The preliminary variable selection step is not included here.

In [20]:
%sql
drop table if exists cleaned_train;

create table cleaned_train as
select MSZoning1, Neighborhood1,  
LotShape1, LandContour1, LandSlope1, PavedDrive1,
Condition21, ExterCond1, BsmtQual1, KitchenQual1,
RoofMatl1, CentralAir1, FireplaceQu1,
GarageType1, GarageFinish1,  GarageQual1, GarageCond1,
SaleCondition1, Functional1,
-- # Numerical below
MSSubClass, LotFrontage, LotArea, OverallQual, OverallCond, 
BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, 1stFlrSF, 2ndFlrSF, GrLivArea,
BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, Fireplaces,
GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, 
PoolArea, YrSold, YearBuilt, YearRemodAdd, MoSold, SalePrice
from traincat

In [21]:
%sql show tables

In [22]:
%scala
val cleaned_train = sqlContext.table("cleaned_train")
display(cleaned_train.describe())

Trials to remove NA values from 'MasVnrArea' column

In [24]:
%scala
val cleaned_train2 = cleaned_train.na.fill(0)
cleaned_train2.registerTempTable("cleaned_train2")
display(cleaned_train2.describe())

In [25]:
%r # could not replace the NAs from MasVnrArea and this r fnc fails also
cleaned_train2[is.na(cleaned_train2)] <- 0

NA removing tentative solution:  'MasVnrArea' variable is not selected from the above table 'cleaned_train'

We're going to have to put all of it into one column of a vector type for Spark MLLib. 

This makes it easy to embed a prediction right in a DataFrame and also makes it very clear as to what is getting passed into the model and what isn't without have to convert it to a numpy array or specify an R formula. 

This also makes it easy to incrementally add new features, simply by adding to the vector. 

In the below case rather than specifically adding them in, I'm going to create a exclusionary group and just remove what is NOT a feature.

In [28]:
%scala
val nonFeatureCols = Array("SalePrice")
val featureCols = cleaned_train2.columns.diff(nonFeatureCols)

In [29]:
%scala
cleaned_train2.printSchema()

Now I'm going to use the VectorAssembler in Apache Spark to Assemble all of these columns into one single vector. To do this I'll have to set the input columns and output column. Then I'll use that assembler to transform the prepped data to my final dataset.

In [31]:
%scala 
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features")
val finalPrep = assembler.transform(cleaned_train2)

val Array(training, testing) = finalPrep.randomSplit(Array(0.7, 0.3))

// Going to cache the data to make sure things stay snappy!
training.cache()
testing.cache()

println(training.count())
println(testing.count())

In [32]:
%scala display(finalPrep)

In [33]:
%scala training.select("MSZoning1").show()

In [34]:
Now we're going to get into the core of Apache Spark MLLib. At a high level, we're going to create an instance of a regressor or classifier, that in turn will then be trained and return a Model type. Whenever you access Spark MLLib you should be sure to import/train on the name of the algorithm you want as opposed to the Model type. 

In [35]:
%scala

import org.apache.spark.ml.regression.LinearRegression

val lrModel = new LinearRegression()
  .setLabelCol("SalePrice")
  .setFeaturesCol("features")
  .setElasticNetParam(0.5)

println("Printing out the model Parameters:")
println("-"*20)
println(lrModel.explainParams)
println("-"*20)

Now finally we can go about fitting our model! You'll see that we're going to do this in a series of steps. First we'll fit it, then we'll use it to make predictions via the transform method. This is the same way you would make predictions with your model in the future however in this case we're using it to evaluate how our model is doing. We'll be using regression metrics to get some idea of how our model is performing, we'll then print out those values to be able to evaluate how it performs.

In [37]:
%scala 
import org.apache.spark.mllib.evaluation.RegressionMetrics
val lrFitted = lrModel.fit(training)

Prediction vs SalePrice

In [39]:
%scala
val holdout = lrFitted
  .transform(testing)
  .selectExpr("SalePrice as SalePrice",
  "prediction as Prediction")
display(holdout)

In [40]:
%scala
val rm = new RegressionMetrics(
  holdout.select("Prediction", "SalePrice").rdd.map(x =>
  (x(0).asInstanceOf[Double], x(1).asInstanceOf[Double])))

println("MSE: " + rm.meanSquaredError)
println("MAE: " + rm.meanAbsoluteError)
println("RMSE Squared: " + rm.rootMeanSquaredError)
println("R Squared: " + rm.r2)
println("Explained Variance: " + rm.explainedVariance + "\n")

2nd method

In [43]:
%scala
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.{Pipeline, PipelineStage}

val rfModel = new RandomForestRegressor()
  .setLabelCol("SalePrice")
  .setFeaturesCol("features")

val paramGrid = new ParamGridBuilder()
  .addGrid(rfModel.maxDepth, Array(5, 10))
  .addGrid(rfModel.numTrees, Array(20, 60))
  .build()
// Note, that this parameter grid will take a long time
// to run in the community edition due to limited number
// of workers available! Be patient for it to run!
// If you want it to run faster, remove some of
// the above parameters and it'll speed right up!

val steps:Array[PipelineStage] = Array(rfModel)

val pipeline = new Pipeline().setStages(steps)

val cv = new CrossValidator() // you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline) // the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid)
  .setEvaluator(new RegressionEvaluator().setLabelCol("SalePrice"))

val pipelineFitted = cv.fit(training)

Now we've trained our model! Let's take a look at which version performed best!

In [45]:
%scala
println("The Best Parameters:\n--------------------")
println(pipelineFitted.bestModel.asInstanceOf[org.apache.spark.ml.PipelineModel].stages(0))
pipelineFitted
  .bestModel.asInstanceOf[org.apache.spark.ml.PipelineModel]
  .stages(0)
  .extractParamMap

In [46]:
%scala
val holdout2 = pipelineFitted.bestModel
  .transform(testing)
  .selectExpr("SalePrice as SalePrice",
  "prediction as Prediction")
display(holdout2)